In [1]:
import pandas
import numpy as np
import yfinance as yf

## MACD (Moving Average Convergence/Divergence) oscillators

In [47]:
# Define the ticker symbol
tickerSymbol = 'GS'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
df = tickerData.history(period='1d', start='2021-1-1', end='2023-1-1')

df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-01-04 00:00:00-05:00,247.795354,248.333624,241.447342,245.939209,3572000,0.0,0.0
2021-01-05 00:00:00-05:00,244.899758,253.827810,243.683986,251.442657,4207100,0.0,0.0
2021-01-06 00:00:00-05:00,256.417146,267.637539,253.456593,265.011078,6383500,0.0,0.0
2021-01-07 00:00:00-05:00,267.071407,274.607379,266.059811,270.672333,4009700,0.0,0.0
2021-01-08 00:00:00-05:00,270.997183,271.257042,264.556357,269.215271,2800800,0.0,0.0


In [48]:
print(df['Close'][-1])
print(df['Close'][0])

332.831298828125
245.939208984375


In [49]:
#Print cumulative last day - first day
print(100*(df['Close'][-1] - df['Close'][0])/df['Close'][0])

35.330718596102514


In [50]:
# Calculate Short Term Exponential Moving Average
ShortEMA = df.Close.ewm(span=12, adjust=False).mean() 
# Calculate Long Term Exponential Moving Average
LongEMA = df.Close.ewm(span=26, adjust=False).mean() 
# Calculate MACD line
MACD = ShortEMA - LongEMA
# Calculate Signal line
signal = MACD.ewm(span=9, adjust=False).mean()

In [51]:
df['MACD'] = MACD
df['Signal Line'] = signal

In [52]:
def buy_sell(signal):
    Buy = []
    Sell = []
    flag = 0

    for i in range(0, len(signal)):
        if signal['MACD'][i] > signal['Signal Line'][i]:
            Sell.append(np.nan)
            if flag != 1:
                Buy.append(signal['Close'][i])
                flag = 1
            else:
                Buy.append(np.nan)
        elif signal['MACD'][i] < signal['Signal Line'][i]:
            Buy.append(np.nan)
            if flag != 0:
                Sell.append(signal['Close'][i])
                flag = 0
            else:
                Sell.append(np.nan)
        else:
            Buy.append(np.nan)
            Sell.append(np.nan)

    return (Buy, Sell)

In [53]:
a = buy_sell(df)
buy_signal = a[0]
sell_signal = a[1]
buy_signal = [x for x in buy_signal if x is not np.nan]
sell_signal = [x for x in sell_signal if x is not np.nan]

In [54]:
print(buy_signal)
print(sell_signal)

[251.44265747070312, 278.56097412109375, 318.93634033203125, 316.1784973144531, 357.5296630859375, 354.8092041015625, 355.7160339355469, 349.6861267089844, 385.5475158691406, 369.0085754394531, 377.84979248046875, 366.4512939453125, 342.861083984375, 312.6794128417969, 313.3633728027344, 297.3000183105469, 287.4701843261719, 287.9289245605469, 293.386474609375, 295.69757080078125]
[268.574951171875, 320.6973571777344, 308.2589111328125, 333.90911865234375, 349.5832824707031, 343.72174072265625, 341.0386047363281, 367.9346923828125, 386.3875732421875, 382.4139099121094, 350.93536376953125, 359.746337890625, 326.7879638671875, 311.7769775390625, 297.4425048828125, 290.7579345703125, 269.1293029785156, 327.2675476074219, 283.3138427734375, 374.4584655761719]


In [55]:
# Assume we buy and sell at the close of the day
stock_Return = []
price_portfolio = []
cash = 100 
for i in range(0, len(sell_signal)):
    shares_to_buy = cash / buy_signal[i]
    new_cash = shares_to_buy * sell_signal[i]
    stock_Return.append(new_cash - cash)
    cash = new_cash
    print(cash)
    
print(stock_Return)

106.81359872406217
122.97070301943579
118.85385959638383
125.51893263882558
122.72917470781348
118.89400015662918
113.98823796704225
119.93677777947993
120.19810425760807
124.56465803927438
115.69185547217053
113.5750426251625
108.25071336266703
107.93828708427282
102.45433024272735
100.19982379564931
93.8069761909499
106.62346304156833
102.96283454899607
130.38762859017868
[6.8135987240621745, 16.157104295373614, -4.116843423051961, 6.6650730424417475, -2.7897579310120904, -3.835174551184309, -4.905762189586923, 5.948539812437673, 0.26132647812814014, 4.366553781666312, -8.872802567103847, -2.1168128470080347, -5.324329262495468, -0.3124262783942129, -5.483956841545464, -2.254506447078043, -6.392847604699412, 12.816486850618432, -3.6606284925722576, 27.424794041182608]


In [56]:
print("Cumulative Return : ", np.sum(stock_Return))
print("Volatility : ", np.std(stock_Return))

Cumulative Return :  30.38762859017868
Volatility :  8.778430762178115


## RSI (Relative Strength Index)  

In [57]:
def calculate_RSI(data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]

    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

df['RSI'] = calculate_RSI(df['Close'], 14)

In [58]:
def buy_sell_RSI(signal, upper=70, lower=30):
    Buy = []
    Sell = []
    flag = 0  # 0: No position, 1: Long position

    for i in range(0, len(signal)):
        if signal['RSI'][i] > upper and flag != 0:
            Sell.append(signal['Close'][i])
            flag = 0  # Change flag to indicate no position
        elif signal['RSI'][i] < lower and flag != 1:
            Buy.append(signal['Close'][i])
            flag = 1  # Change flag to indicate long position

    return (Buy, Sell)

In [59]:
a = buy_sell_RSI(df)
buy_signal = a[0]
sell_signal = a[1]
buy_signal = [x for x in buy_signal if x is not np.nan]
sell_signal = [x for x in sell_signal if x is not np.nan]

In [60]:
# Assume we buy and sell at the close of the day
stock_Return = []
cash = 100 
for i in range(0, len(sell_signal)):
    shares_to_buy = cash / buy_signal[i]
    new_cash = shares_to_buy * sell_signal[i]
    stock_Return.append(new_cash - cash)
    cash = new_cash
    print(cash)
    
print(stock_Return)

101.37740785390719
117.02532370500572
[1.3774078539071866, 15.647915851098531]


In [61]:
print("Cumulative Return : ", np.sum(stock_Return))
print("Volatility : ", np.std(stock_Return))

Cumulative Return :  17.025323705005718
Volatility :  7.135253998595672
